In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Import other libraries
import os
import sys

# Import our own libraries
sys.path.append("../lib/")
import eyegaze as eg

In [ ]:
# データパスの設定
group_letter = "A"  # グループレター: "A" または "B"
participant_id = "P001"
phase = "pre"
base_dir = f"../../data/input/{group_letter}/{participant_id}/{phase}"

# 自動検出: eye_trackingディレクトリ内のタイムスタンプフォルダ（ソートして最新を取得）
eye_tracking_base = os.path.join(base_dir, "eye_tracking")
timestamp_dirs = sorted([d for d in os.listdir(eye_tracking_base) 
                         if os.path.isdir(os.path.join(eye_tracking_base, d))])
eye_tracking_dir = os.path.join(eye_tracking_base, timestamp_dirs[-1])  # 最新

# イベントログ（ソートして最新を取得）
log_dir = os.path.join(base_dir, "logs")
event_files = sorted([f for f in os.listdir(log_dir) if f.endswith('.jsonl')])
event_log_path = os.path.join(log_dir, event_files[-1])  # 最新

# 出力先
output_base = f"../../data/output/{group_letter}/{participant_id}/{phase}"
working_base = f"../../data/working/{group_letter}/{participant_id}/{phase}"

print(f"Group: {group_letter}, Participant: {participant_id}, Phase: {phase}")
print(f"Eye tracking dir: {eye_tracking_dir}")
print(f"Event log: {event_log_path}")
print(f"Output: {output_base}")

In [ ]:
# ========== 頭部位置補正の設定 ==========
APPLY_HEAD_CORRECTION = False     # 頭部位置補正を適用するか
CORRECTION_METHOD = "trackbox"   # "geometric" または "trackbox"

# --- 共通設定 ---
SCREEN_WIDTH_PX = 1920
SCREEN_HEIGHT_PX = 1080

# --- geometric方式用 ---
USE_AVERAGE_REFERENCE = True
CORRECT_Y = True
CALIBRATION_HEAD_X = 0.0
CALIBRATION_HEAD_Y = 0.0
SCREEN_WIDTH_MM = 509.0
SCREEN_HEIGHT_MM = 287.0

# --- trackbox方式用 ---
CORRECTION_FACTOR_X = 500.0
CORRECTION_FACTOR_Y = -100.0
CALIBRATION_CENTER_X = 0.5
CALIBRATION_CENTER_Y = 0.5
# ==========================================

# データ読み込み（統合API - phaseで切り替え）
# 使用可能なフェーズ: "pre", "posttest", "training1", "training2"
segments = eg.readTobiiData(
    eye_tracking_dir=eye_tracking_dir,
    event_log_path=event_log_path,
    phase=phase,
    apply_head_correction=APPLY_HEAD_CORRECTION,
    correction_method=CORRECTION_METHOD,
    # geometric用
    use_average_reference=USE_AVERAGE_REFERENCE,
    correct_y=CORRECT_Y,
    calibration_head_x=CALIBRATION_HEAD_X,
    calibration_head_y=CALIBRATION_HEAD_Y,
    screen_width_mm=SCREEN_WIDTH_MM,
    screen_height_mm=SCREEN_HEIGHT_MM,
    screen_width_px=SCREEN_WIDTH_PX,
    screen_height_px=SCREEN_HEIGHT_PX,
    # trackbox用
    correction_factor_x=CORRECTION_FACTOR_X,
    correction_factor_y=CORRECTION_FACTOR_Y,
    calibration_center_x=CALIBRATION_CENTER_X,
    calibration_center_y=CALIBRATION_CENTER_Y
)

correction_str = f"{CORRECTION_METHOD}" if APPLY_HEAD_CORRECTION else "none"
print(f"Total segments: {len(segments)} (head correction: {correction_str})")
for seg in segments:
    # フェーズに応じた表示
    if 'event_type' in seg:
        print(f"  {seg['image_number']}: {seg['event_type']:30s} | {len(seg['data']):5d} samples, {seg['duration']:.2f}s")
    else:
        print(f"  {seg['passage_id']}: {len(seg['data'])} samples, {seg['duration']:.2f}s, image: {seg['image_number']}")

In [ ]:
# 頭部位置補正の比較表示（補正なしデータを読み込んで比較）
if APPLY_HEAD_CORRECTION:
    segments_original = eg.readTobiiData(
        eye_tracking_dir=eye_tracking_dir,
        event_log_path=event_log_path,
        phase=phase,
        apply_head_correction=False  # 補正なし
    )
    
    # サンプルセグメントで比較表示
    sample_idx = 0
    original_data = segments_original[sample_idx]['data']
    corrected_data = segments[sample_idx]['data']
    image_path = segments[sample_idx]['image_path']
    
    print(f"Comparing: {segments[sample_idx].get('passage_id') or segments[sample_idx].get('event_type')}")
    print(f"Correction method: {CORRECTION_METHOD}")
    print(f"Original gaze_x range: {original_data[:, 1].min():.1f} - {original_data[:, 1].max():.1f}")
    print(f"Corrected gaze_x range: {corrected_data[:, 1].min():.1f} - {corrected_data[:, 1].max():.1f}")
    print(f"X shift (mean): {corrected_data[:, 1].mean() - original_data[:, 1].mean():.1f} px")
    print(f"Y shift (mean): {corrected_data[:, 2].mean() - original_data[:, 2].mean():.1f} px")
    
    eg.plotGazeCorrectionComparison(
        original_data, corrected_data, 
        bg_image=image_path,
        title=f"Head Position Correction ({CORRECTION_METHOD})"
    )
else:
    print("Head correction is disabled. Set APPLY_HEAD_CORRECTION=True to see comparison.")

In [ ]:
# サンプル可視化（1つのセグメント）
sample_seg = segments[0]
data = sample_seg['data']
image_path = sample_seg['image_path']

print(f"Analyzing: {sample_seg['passage_id']}")

# 生の視線データ（スキャンパス）
eg.plotScanPath(data[:, 1], data[:, 2], 
                np.array([50.0 for x in data]), 
                bg_image=image_path)

# Fixation検出（瞳孔径も含める）
fx = eg.detectFixations(data[:, 0], data[:, 1], data[:, 2], P=data[:, 3],
                        min_concat_gaze_count=9, 
                        min_fixation_size=20, 
                        max_fixation_size=40)

# Fixationベースのスキャンパス（秒単位なのでduration_scale=1000）
eg.plotScanPath(fx[:, 1], fx[:, 2], fx[:, 3], 
                bg_image=image_path, duration_scale=1000)

# ヒートマップ
eg.plotHeatmap(fx[:, 1], fx[:, 2], fx[:, 3], 
               bg_image=image_path)

In [ ]:
# 全セグメントの一括処理
for segment in segments:
    passage_id = segment['passage_id']
    img_num = segment['image_number']
    data = segment['data']
    image_path = segment['image_path']
    
    print(f"Processing {passage_id} (image {img_num})...")
    
    # 出力ディレクトリ作成
    for subdir in ['gaze_raw', 'scan_path', 'heatmap']:
        os.makedirs(os.path.join(output_base, subdir), exist_ok=True)
    os.makedirs(os.path.join(working_base, 'fixation'), exist_ok=True)
    
    # 生の視線データ
    eg.plotScanPath(
        data[:, 1], data[:, 2], 
        np.array([50.0 for x in data]),
        bg_image=image_path, 
        save_path=os.path.join(output_base, 'gaze_raw', f"{img_num}.png")
    )
    
    # Fixation検出（瞳孔径も含める）
    fx = eg.detectFixations(data[:, 0], data[:, 1], data[:, 2], P=data[:, 3],
                            min_concat_gaze_count=9,
                            min_fixation_size=20,
                            max_fixation_size=40)
    
    # 注視点が検出されなかった場合はスキップ
    if fx.shape[0] == 0:
        print(f"  Skipped: {img_num} (no fixations detected)")
        continue
    
    # Fixationデータ保存
    np.savetxt(
        os.path.join(working_base, 'fixation', f"{img_num}.csv"),
        fx, 
        delimiter=',', 
        header='timestamp,x,y,duration,saccade_length,saccade_angle,saccade_speed,pupil_diameter',
        comments=''
    )
    
    # スキャンパス（秒単位なのでduration_scale=1000）
    eg.plotScanPath(
        fx[:, 1], fx[:, 2], fx[:, 3],
        bg_image=image_path,
        save_path=os.path.join(output_base, 'scan_path', f"{img_num}.png"),
        duration_scale=1000
    )
    
    # ヒートマップ
    eg.plotHeatmap(
        fx[:, 1], fx[:, 2], fx[:, 3],
        bg_image=image_path,
        save_path=os.path.join(output_base, 'heatmap', f"{img_num}.png")
    )

print("All segments processed successfully!")

In [ ]:
# 統計情報の出力
stats = []
for segment in segments:
    fx = eg.detectFixations(segment['data'][:, 0], 
                           segment['data'][:, 1], 
                           segment['data'][:, 2],
                           P=segment['data'][:, 3],
                           min_concat_gaze_count=9,
                           min_fixation_size=50,
                           max_fixation_size=80)
    
    stat = {
        'image_number': segment['image_number'],
        'duration_sec': segment['duration'],
        'raw_samples': len(segment['data']),
        'fixation_count': len(fx),
        'total_fixation_duration': fx[:, 3].sum() if len(fx) > 0 else 0,
        'mean_fixation_duration': fx[:, 3].mean() if len(fx) > 0 else 0,
        'mean_pupil_diameter': fx[:, 7].mean() if len(fx) > 0 else 0
    }
    
    # フェーズに応じて追加フィールド
    if 'passage_id' in segment:
        stat['passage_id'] = segment['passage_id']
    if 'event_type' in segment:
        stat['event_type'] = segment['event_type']
    if 'analog_id' in segment:
        stat['analog_id'] = segment['analog_id']
    
    stats.append(stat)

stats_df = pd.DataFrame(stats)
print(stats_df)

# CSV保存
stats_df.to_csv(os.path.join(output_base, 'statistics.csv'), index=False)